In [1]:
import torch

In [2]:
from torch.distributed.checkpoint.state_dict import get_state_dict

In [8]:
import torch.nn as nn
class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = nn.Linear(16, 16)
        self.relu = nn.ReLU()
        self.net2 = nn.Linear(16, 8)

    def forward(self, x):
        return self.net2(self.relu(self.net1(x)))
model = ToyModel()

In [9]:
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
model_sd, optim_sd = get_state_dict(model, optim)

In [15]:
import torch.distributed.checkpoint as dcp
CHECKPOINT_DIR = "."
dcp.save(model_sd, checkpoint_id=CHECKPOINT_DIR)

Metadata(state_dict_metadata={'net1.weight': TensorStorageMetadata(properties=TensorProperties(dtype=torch.float32, layout=torch.strided, requires_grad=False, memory_format=torch.contiguous_format, pin_memory=False), size=torch.Size([16, 16]), chunks=[ChunkStorageMetadata(offsets=torch.Size([0, 0]), sizes=torch.Size([16, 16]))]), 'net1.bias': TensorStorageMetadata(properties=TensorProperties(dtype=torch.float32, layout=torch.strided, requires_grad=False, memory_format=torch.contiguous_format, pin_memory=False), size=torch.Size([16]), chunks=[ChunkStorageMetadata(offsets=torch.Size([0]), sizes=torch.Size([16]))]), 'net2.weight': TensorStorageMetadata(properties=TensorProperties(dtype=torch.float32, layout=torch.strided, requires_grad=False, memory_format=torch.contiguous_format, pin_memory=False), size=torch.Size([8, 16]), chunks=[ChunkStorageMetadata(offsets=torch.Size([0, 0]), sizes=torch.Size([8, 16]))]), 'net2.bias': TensorStorageMetadata(properties=TensorProperties(dtype=torch.floa

In [14]:
!pwd

4229.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/data/users/howardhuang


In [17]:
sd = {"model": model}
dcp.load(sd, checkpoint_id=CHECKPOINT_DIR)

CheckpointException: CheckpointException ranks:dict_keys([0])
Traceback (most recent call last): (RANK 0)
  File "/home/howardhuang/local/pytorch/torch/distributed/checkpoint/utils.py", line 192, in reduce_scatter
    local_data = map_fun()
  File "/home/howardhuang/local/pytorch/torch/distributed/checkpoint/logger.py", line 87, in wrapper
    result = func(*args, **kwargs)
  File "/home/howardhuang/local/pytorch/torch/distributed/checkpoint/state_dict_loader.py", line 223, in local_step
    local_plan = planner.create_local_plan()
  File "/home/howardhuang/local/pytorch/torch/distributed/checkpoint/default_planner.py", line 334, in create_local_plan
    return create_default_local_load_plan(
  File "/home/howardhuang/local/pytorch/torch/distributed/checkpoint/default_planner.py", line 455, in create_default_local_load_plan
    raise RuntimeError(f"Missing key in checkpoint state_dict: {fqn}.")
RuntimeError: Missing key in checkpoint state_dict: model.net1.weight.


In [7]:
from torch.distributed.checkpoint._in_memory_checkpoint import my_function

my_function()

ImportError: cannot import name 'my_function' from 'torch.distributed.checkpoint._in_memory_checkpoint' (/home/howardhuang/local/pytorch/torch/distributed/checkpoint/_in_memory_checkpoint.py)

In [ ]:
t = torch.Tensor([1, 2, 3])
t.stride()

(1,)

In [ ]:
import mmap
import os

# Define the file path and size
file_path = 'persistent_storage.dat'
file_size = 1024  # 1 KB for demonstration

# Create a file and set its size
with open(file_path, 'wb') as f:
    f.write(b'\x00' * file_size)

# Open the file for reading and writing
with open(file_path, 'r+b') as f:
    # Memory-map the file
    mm = mmap.mmap(f.fileno(), file_size)

    # Write data to the memory-mapped file
    mm[0:12] = b'Hello Worlda'

    # Read data from the memory-mapped file
    print(mm[0:15].decode('utf-8'))  # Output: Hello World

    # Close the memory-mapped file
    mm.close()

# At this point, the data is saved to 'persistent_storage.dat' on disk

# To demonstrate persistence, open the file again in a new process or later
with open(file_path, 'r+b') as f:
    mm = mmap.mmap(f.fileno(), file_size)

    # Read the persisted data
    print(mm[0:11].decode('utf-8'))  # Output: Hello World

    mm.close()

Hello Worlda   
Hello World


In [ ]:
ls

__0_0.distcp  dotsync-home/  persistent_storage.dat
bento-cache/  notebooks/     scratch/
